---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="d0838da2" -->
# Table des matières
- [Exemples de régressions avec données aberrantes](#exemples-de-régressions-avec-données-aberrantes)
  - [Que sont les valeurs aberrantes?](#que-sont-les-valeurs-aberrantes)
  - [Initialisations](#initialisations)
  - [Exemple I: recalage linéaire avec donnée aberrante](#exemple-i-recalage-linéaire-avec-donnée-aberrante)
      - [Génération des données bruitées](#génération-des-données-bruitées)
      - [Recalage linéaire des moindres carrés sans valeur aberrante](#recalage-linéaire-des-moindres-carrés-sans-valeur-aberrante)
      - [Ajout d'une valeur aberrante à la position (9, 2)](#ajout-dune-valeur-aberrante-à-la-position-9-2)
      - [Recalage linéaire des moindres carrés avec valeur aberrante](#recalage-linéaire-des-moindres-carrés-avec-valeur-aberrante)
      - [Calcul des signaux vrais et recalés en utilisant les coefficients vrais et estimés](#calcul-des-signaux-vrais-et-recalés-en-utilisant-les-coefficients-vrais-et-estimés)
      - [Affichage des résultats du recalage avec et sans la valeur aberrante](#affichage-des-résultats-du-recalage-avec-et-sans-la-valeur-aberrante)
  - [Exemple II: recalage quadratique avec donnée aberrante](#exemple-ii-recalage-quadratique-avec-donnée-aberrante)
      - [Génération des données bruitées](#génération-des-données-bruitées-1)
      - [Recalage quadratique des moindres carrés sans valeur aberrante](#recalage-quadratique-des-moindres-carrés-sans-valeur-aberrante)
      - [Ajout d'une valeur aberrante à la position (9, -1)](#ajout-dune-valeur-aberrante-à-la-position-9--1)
      - [Recalage quadratique des moindres carrés avec valeur aberrante](#recalage-quadratique-des-moindres-carrés-avec-valeur-aberrante)
      - [Calcul des signaux vrais et recalés en utilisant les coefficients vrais et estimés](#calcul-des-signaux-vrais-et-recalés-en-utilisant-les-coefficients-vrais-et-estimés-1)
      - [Affichage des résultats du recalage avec et sans la valeur aberrante](#affichage-des-résultats-du-recalage-avec-et-sans-la-valeur-aberrante-1)
      - [En résumé](#en-résumé)
  - [Exercice](#exercice)
<!-- #endregion -->

<!-- #region id="7cc01bb5" -->
---
# Exemples de régressions avec données aberrantes
---
<!-- #endregion -->

<!-- #region id="56e9763d" -->
<div align="center">
    <img src= "../images/professor-in-green.jpeg"  width="300" />
    <div>
    <font size="0.5">Image Source: https://freesvg.org/professor-in-green-suit/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="ea28f1cf" -->
Dans ce module, on montre l'effet des valeurs aberrantes (*outliers*) sur les performances
de régressions basées sur la **méthode des moindres carrés**.

On applique les principes de la régression 1-D à des modèles linéaires dans les coefficients, où la réponse y
est une fonction de la variable x et des coefficients $\Theta$

$$y=f(x, \Theta)$$

Supposons que l'on ait un jeu de données $(x^{(i)},y^{(i)},\sigma^{(i)})$ où les $\sigma^{(i)}$ sont les erreurs de mesures associées aux $y^{(i)}$. L'indice $i$ représente le numéro d'une donnée tel que $1\le i \le n$, où $n$ est le nombre de données. Dans la majorité des cas, les erreurs de mesures sont similaires et on peut les ignorer dans l'analyse.

Selon la méthode des moindres carrés, on peut estimer les valeurs des coefficients $\Theta$ en minimisant la fonction suivante

$$S(\Theta) = \sum\limits_{i=1}^{n}(y^{(i)}-f(x^{(i)},\Theta))^{2}$$

Comme on l'a vu dans un module précédent, son minimum se calcule par différenciation : $\nabla S(\Theta) = 0$. La solution de cette équation différentielle constitue l'ensemble des coefficients $\Theta$ recherchés.

La fonction [Polynomial.fit](https://numpy.org/doc/stable/reference/generated/numpy.polynomial.polynomial.Polynomial.fit.html#numpy.polynomial.polynomial.Polynomial.fit) de la librairie Numpy calcule pour nous les coefficients $\Theta$ pour des polynômes de degrés arbitraires. Nous allons l'utiliser dans ce module.

Nous allons découvrir à quel point les valeurs aberrantes sont le talon d'Achille de l'approche par les moindres carrés. Les principes expliqués dans ce module se généralisent en plusieurs dimensions,
il est donc intéressant de les présenter dans la situation la plus simple.

<!-- #endregion -->

<!-- #region id="5fadba4e" -->
## Que sont les valeurs aberrantes?
<!-- #endregion -->

<!-- #region id="f85add68" -->
<div align="center">
    <img src= "../images/men-in-black-orange.png"  width="300" />
    <div>
    <font size="0.5">Image Source: https://freesvg.org/radacina-men-in-black-orange/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="cf91cfe6" -->
Comme leur nom l'indique, les valeurs aberrantes ont une distribution qui se démarque de l'ensemble des
données que l'on veut analyser. En fait, l'observation des données est une étape essentielle à
l'analyse exploratoire des données (*exporatory data analysis*). Il ne faut pas s'en tenir à des
mesures statistiques de la distribution des données; il faut aussi regarder celles-ci. C'est une erreur courante.

Il arrive régulièrement que l'on utilise des bases de données avec des valeurs manquantes; elles sont généralement
identifiées par des NaN (*not a number*) ou simplement remplacées par des zéros. Le dernier cas est
plus problématique si les données sont utilisées telles quelles dans l'analyse. Elles deviennent alors
des valeurs aberrantes, par rapport aux autres du même jeu de données. C'est une des raisons pour lesquelles
le prétraitement des données est une étape essentielle dans la préparation des données. On en discute dans
un des modules portant sur le sujet.

Les valeurs aberrantes proviennent de plusieurs sources dont en voici quelques unes:


- erreurs de lecture lors de la prise de données,
- valeurs mal entrées lors de l'enregistrement des données,
- valeurs utilisant des échelles différentes (mm, cm, m, km),
- mesures perturbées par un bruit instrumental important,
- valeurs non attendues, mais bien réelles (découvertes potentielles!)


La figure suivante montre un exemple typique de valeur aberrante, en rouge, superposée à une
courbe de décroissance exponentielle en fonction du temps.
<!-- #endregion -->

<!-- #region id="e66ee85a" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/regression-outlier.png"  width="350" />
    <div>
    <font size="0.5">Image Source: https://www.graphpad.com/guides/prism/8/curve-fitting/reg_graphing_outliers.htm</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="36048a79" -->
Attention toutefois, les valeurs aberrantes ne sont pas toujours ce que l'on croit. Le panneau de gauche
dans la figure suivante montre deux valeurs aberrantes en rouge. Le point rouge dans le panneau de droite n'est
pas une valeur aberrante, mais plutôt un point de bascule (*leverage point*). Ce type de point,
souvent valide, peut avoir un effet disproportionné sur le recalage des données, mais nous n'aborderons pas ce sujet.
<!-- #endregion -->

<!-- #region id="1df218cf" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/outliers-leverage-point.png"  width="500" />
    <div>
    <font size="0.5">Image Source: https://marcellovictorino.github.io/post/Linear-Regression-Part2/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="d5bbde81" -->
Dans ce module, nous allons voir comment varient les résultats de régressions
basées sur la **méthode des moindres carrés** lorsqu'on introduit une valeur aberrante. L'effet est d'autant plus
prononcé que le nombre d'anomalies est grand.

<!-- #endregion -->

<!-- #region id="dee8d061" -->
## Initialisations
<!-- #endregion -->



In [ ]:
%matplotlib inline

import numpy as np
from numpy.polynomial.polynomial import polyfit, polyval
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True)

# Pour la reproductibilité des résultats
seed = 42
np.random.seed(seed)



<!-- #region id="234bea14" -->
Définissons deux fonctions utiles pour les exemples qui suivent.
La première fonction génère des données sous la forme d'un polynôme additionné de bruit.
La seconde fonction affiche les résultats de régressions avec et sans valeur aberrante
afin de comparer leurs performances.
<!-- #endregion -->



In [ ]:
def genere_donnees(coeffs, sigma, x_min, x_max, n):

    # Génération de n positions x aléatoires
    x = np.random.uniform(x_min, x_max, n)
    x = np.sort(x)

    # Génération d'un polynôme de coefficients donnés et ajout de
    # bruit gaussien d'écart-type (ou niveau de bruit) sigma
    y = np.polyval(coeffs, x) + np.random.normal(0.0, sigma, n)

    return x, y


def affiche_resultats():
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12, 5))

    # Cas sans valeur aberrante
    ax1.plot(x, y_vrai, color='yellow', linewidth=2)
    ax1.plot(x, y1_pred, color='red', linewidth=2)
    ax1.plot(x1, y1, 'o', color='black', markersize=5)
    ax1.set_ylabel('y', rotation=0, fontsize=18)
    ax1.set_xlabel('x', fontsize=18)
    ax1.set_title('Sans valeur aberrante', fontsize=18)
    ax1.legend(['Vrai modèle', 'Modèle recalé'], loc='upper left')
    ax1.set_facecolor('#BBBBFF')

    # Cas avec valeur aberrante
    ax2.plot(x, y_vrai, color='yellow', linewidth=2)
    ax2.plot(x, y2_pred, color='red', linewidth=2)
    ax2.plot(x2[:-1], y2[:-1], 'o', color='black', markersize=5)
    ax2.plot(x2[-1], y2[-1], '*', color='green', markersize=10)
    ax2.set_ylabel('y', rotation=0, fontsize=18)
    ax2.set_xlabel('x', fontsize=18)
    ax2.set_title('Avec valeur aberrante', fontsize=18)
    ax2.legend(['Vrai modèle', 'Modèle recalé'], loc='upper left')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.99)
    ax2.set_facecolor('#BBBBFF')



<!-- #region id="4c2cb94d" -->
## Exemple I: recalage linéaire avec donnée aberrante

Dans cet exemple, nous générons une droite et ajoutons du bruit aux données en y. Nous
effectuons ensuite un recalage linéaire aux données $(x,y)$ et comparons avec la droite originale.

Puis, nous ajoutons une valeur aberrante à l'ensemble de données original et refaisons
l'analyse afin de voir l'effet de cette valeur sur la qualité des résultats.

<!-- #endregion -->

<!-- #region id="c7706e6a" -->
#### Génération des données bruitées
<!-- #endregion -->



In [ ]:
# Coefficients de la droite
coeffs = [1, 2]
degre = len(coeffs) - 1

# Génération des données
npts = 20
sigma = 1.0
x_min = -2.5
x_max = 10.0
x1, y1 = genere_donnees(coeffs, sigma, x_min, x_max, npts)



<!-- #region id="01a4eb44" -->
#### Recalage linéaire des moindres carrés sans valeur aberrante
<!-- #endregion -->



In [ ]:
c1 = np.polyfit(x1, y1, degre)



<!-- #region id="00b703d2" -->
#### Ajout d'une valeur aberrante à la position (9, 2)
<!-- #endregion -->



In [ ]:
x2 = np.append(x1, [9], axis=0)
y2 = np.append(y1, [2], axis=0)



<!-- #region id="c566b956" -->
#### Recalage linéaire des moindres carrés avec valeur aberrante
<!-- #endregion -->



In [ ]:
c2 = np.polyfit(x2, y2, degre)



<!-- #region id="88ecd7fb" -->
#### Calcul des signaux vrais et recalés en utilisant les coefficients vrais et estimés
<!-- #endregion -->



In [ ]:
x = np.linspace(x_min, x_max, 100)

y_vrai = np.polyval(coeffs, x)
y1_pred = np.polyval(c1, x)
y2_pred = np.polyval(c2, x)



<!-- #region id="61f6ad49" -->
#### Affichage des résultats du recalage avec et sans la valeur aberrante
<!-- #endregion -->



In [ ]:
affiche_resultats()



<!-- #region id="ccf22990" -->
Dans chaque panneau, la courbe en jaune montre les prédictions du modèle original et en rouge
les prédictions du modèle recalé à partir des données bruitées.

Dans le panneau de gauche, les droites originale et recalée sont presque identiques. Ce n'est pas
le cas du panneau de droite contenant une valeur aberrante identifiée par l'étoile verte en bas à droite.

L'effet de la valeur aberrante est clairement visible. La droite recalée est 'attirée' par la
valeur aberrante. L'effet est d'autant plus important que le point problématique est situé
loin de la courbe idéale et près des valeurs extrèmes en x.

<!-- #endregion -->

<!-- #region id="a9486008" -->
## Exemple II: recalage quadratique avec donnée aberrante

On reprend l'exemple précédent, mais avec un modèle quadratique, c'est-à-dire, un polynôme de degré 2.

#### Génération des données bruitées
<!-- #endregion -->



In [ ]:
# Coefficients de la quadratique
coeffs = [0.03, 0.2, -2]
degre = len(coeffs) - 1

# Génération des données
npts = 20
sigma = 0.3
x1, y1 = genere_donnees(coeffs, sigma, x_min, x_max, npts)



<!-- #region id="f87c90af" -->
#### Recalage quadratique des moindres carrés sans valeur aberrante
<!-- #endregion -->



In [ ]:
c1 = np.polyfit(x1, y1, degre)



<!-- #region id="e9819423" -->
#### Ajout d'une valeur aberrante à la position (9, -1)
<!-- #endregion -->



In [ ]:
x2 = np.append(x1, [9], axis=0)
y2 = np.append(y1, [-1], axis=0)



<!-- #region id="70f2d6d4" -->
#### Recalage quadratique des moindres carrés avec valeur aberrante
<!-- #endregion -->



In [ ]:
c2 = np.polyfit(x2, y2, degre)



<!-- #region id="eababa62" -->
#### Calcul des signaux vrais et recalés en utilisant les coefficients vrais et estimés
<!-- #endregion -->



In [ ]:
x = np.linspace(x_min, x_max, 100)
y_vrai = np.polyval(coeffs, x)
y1_pred = np.polyval(c1, x)
y2_pred = np.polyval(c2, x)



<!-- #region id="e82bd077" -->
#### Affichage des résultats du recalage avec et sans la valeur aberrante
<!-- #endregion -->



In [ ]:
affiche_resultats()



<!-- #region id="6e0fa99f" -->
Dans le panneau de gauche, les quadratiques originale et recalée sont presque identiques. Dans le
panneau de droite, la quadratique recalée est à nouveau 'attirée' par la valeur aberrante et déformée
au point de ressembler à une droite de régression.

#### En résumé


- Les régressions basées sur la méthode des moindres carrés sont sensibles aux valeurs aberrantes.
- L'effet est d'autant plus important que l'ordre d'un polynôme est élevé.
- Les valeurs aberrantes ont comme effet d'attirer la courbe recalée vers eux.


La présence de valeurs aberrantes dans les données est toujours une chose à vérifier avant d'aller
plus loin dans l'analyse. C'est généralement effectué lors de l'étape de prétraitement des données
qui sera abordée dans un prochain module.
<!-- #endregion -->

<!-- #region id="963f0eb9" -->
## Exercice

Augmentez le nombre de données, p. ex. de 20 à 75 et exécutez à nouveau le code de la partie II. Que remarquez-vous
par rapport à l'effet de la valeur aberrante? **Réponse: l'effet de la valeur aberrante diminue lorsque le nombre de données augmente. Il est toujours préférable d'éliminer les valeurs aberrantes dans nos données, mais si le nombre de données est très grand, les valeurs aberrantes ont peu d'effet.**
<!-- #endregion -->



In [ ]:
# SOLUTION

npts = 75
# Génération des données bruitées
sigma = 0.3

# Coefficients de la quadratique
coeffs = [0.03, 0.2, -2]
degre = len(coeffs) - 1

# Génération des données
x1, y1 = genere_donnees(coeffs, sigma, x_min, x_max, npts)

# Recalage quadratique des moindres carrés sans valeur aberrante
c1 = np.polyfit(x1, y1, degre)

# Ajout d'une valeur aberrante à la position (9, -1)
x2 = np.append(x1, [9], axis=0)
y2 = np.append(y1, [-1], axis=0)

# Recalage quadratique des moindres carrés avec valeur aberrante
c2 = np.polyfit(x2, y2, degre)

# Calcul des signaux vrais et recalés en utilisant les coefficients vrais et estimés
x = np.linspace(x_min, x_max, 100)
y_vrai = np.polyval(coeffs, x)
y1_pred = np.polyval(c1, x)
y2_pred = np.polyval(c2, x)

# Affichage des résultats du recalage avec et sans la valeur aberrante
affiche_resultats()
